In [5]:

import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import random as rn
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, GlobalAvgPool2D
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model


In [6]:

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def create_tfrecord(images_path, labels, tfrecord_filename):
    with tf.io.TFRecordWriter(tfrecord_filename) as writer:
        for label in labels:
            path = images_path.format(label)
            folder_data = os.listdir(path)
            for image_path in folder_data:
                image = cv2.imread(path + image_path, cv2.IMREAD_COLOR)
                image_resized = cv2.resize(image, (224, 224))
                img_bytes = cv2.imencode('.jpg', image_resized)[1].tostring()
                
                label_encoded = labels.index(label)

                feature = {
                    'image': _bytes_feature(img_bytes),
                    'label': _int64_feature(label_encoded)
                }
                
                example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
                writer.write(example_proto.SerializeToString())

# Define paths and labels
labels = os.listdir('data/train')
train_path = 'data/train/{0}/'
test_path = 'data/valid/{0}/'
test_label = os.listdir('data/valid')

# Creating TFRecords for training and testing data
create_tfrecord(train_path, labels, "train.tfrecord")
create_tfrecord(test_path, test_label, "test.tfrecord")


C:\Users\Kartabya Krishna\AppData\Local\Temp\ipykernel_28968\3039545816.py:19: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  img_bytes = cv2.imencode('.jpg', image_resized)[1].tostring()


In [ ]:

def decode_tfrecord(example_proto):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.io.decode_jpeg(example['image'], channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0,1]
    label = example['label']
    return image, label

def load_dataset(tfrecord_name, batch_size):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_name)
    dataset = raw_dataset.map(decode_tfrecord)
    dataset = dataset.shuffle(1024).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

batch_size = 32
train_dataset = load_dataset("train.tfrecord", batch_size)
test_dataset = load_dataset("test.tfrecord", batch_size)


In [ ]:
num = []
for label in labels:
    k = 0
    print('\n', label.upper())
    for image_path in x_data:
        k = k+1
    num.append(k)
    print('there are ', k,' images in ', label, 'class')

In [ ]:
plt.figure(figsize = (8,8))
plt.bar(labels, num)
plt.title('NUMBER OF IMAGES CONTAINED IN EACH CLASS')
plt.xlabel('classes')
plt.xticks(rotation=90)
plt.ylabel('count')
plt.show()

In [ ]:

fig, ax = plt.subplots(5, 2)
fig.set_size_inches(15, 15)

# Fetch a batch of data from the training dataset
for images, labels in train_dataset.take(1):
    images = images.numpy()
    labels = labels.numpy()

    for i in range(5):
        for j in range(2):
            l = rn.randint(0, batch_size - 1)
            ax[i, j].imshow(images[l])
            ax[i, j].set_title('Label: ' + str(labels[l]))

plt.tight_layout()


In [ ]:
x_data = np.array(x_data)

y_data = np.array(y_data)

print('the shape of X is: ', x_data.shape, 'and that of Y is: ', y_data.shape)

In [ ]:
le=LabelEncoder()
Y=le.fit_transform(y_data)
Y=to_categorical(Y,num_cls)
x_data = x_data/255   #standarization 
Y.shape

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

def build_model(input_shape=(224, 224, 3)):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    
    for layer in base_model.layers[:]:
        layer.trainable = False
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu', kernel_regularizer=l2())(x)
    predictions = Dense(38, activation='softmax')(x)
    
    # Construct the final model
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Compile the model with specified optimizer and loss
    model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Build the model
model = build_model()
model.summary() 

In [ ]:
test_label = os.listdir('data/valid')
print(test_label)

In [ ]:
x_test =[]
y_test = []

for label in test_label:
    path = test_path.format(label)
    folder_data = os.listdir(path)
    for image_path in folder_data:
        image = cv2.imread(path+image_path,cv2.IMREAD_COLOR)
        image_resized = cv2.resize(image, (224,224))
        x_test.append(np.array(image_resized))
        y_test.append(label)

In [ ]:
x_test = np.array(x_test)
x_test = x_test/255
y_test = np.array(y_test)
le = LabelEncoder()
y = le.fit_transform(y_test)
y = to_categorical(y,num_cls)

In [ ]:
x_test.shape,y.shape

In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=7, verbose=1, mode='auto')

In [ ]:
history = model.fit(x_data,Y,epochs = 50, validation_data = (x_test,y),
                    verbose = 1)

In [ ]:
    fig=plt.figure()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])    # plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    fig.savefig("accuracy.png")
    # # summarize history for loss
    fig=plt.figure()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    fig.savefig("loss.png")

In [ ]:
model.save('plantIncepV3.hdf5')
# model.load_weights('plantIncepV3.hdf5')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# # 1. Load the trained model
model = load_model('plantIncepV3.hdf5')

# 2. Preprocess the image
def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.  # Assuming your model uses the same normalization
    return img_array

img_path = 'bs123.png'
img_array = preprocess_image(img_path)

# 3. Predict
predictions = model.predict(img_array)

predicted_class = np.argmax(predictions[0])

# 4. Postprocess (if needed)
# If you have class names, you can map the predicted_class index to its name
class_names = ['bs', 'ycv']  # Replace with your actual class names
print(f"Predicted class: {class_names[predicted_class]}")
